In [1]:
!pip install dateparser

     |████████████████████████████████| 288 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 2.5 MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 339 kB 3.7 MB/s eta 0:00:01
  Created wheel for backports.zoneinfo: filename=backports.zoneinfo-0.2.1-cp37-cp37m-macosx_10_13_x86_64.whl size=35143 sha256=6af688bdcfc7ae073719a725aed0c7d227f7caeebcc0c92853628c2c37748515
  Stored in directory: /Users/evgondar/Library/Caches/pip/wheels/ce/bb/ad/1f44f43f7c4fce0a96dbe348c4acdf4fe2481496933357be62
Successfully built backports.zoneinfo
You should consider upgrading via the '/Users/evgondar/.pyenv/versions/3.7.0/bin/python3.7 -m pip install --upgrade pip' command.


## Parse JACS for latest articles

In [2]:
import requests
from urllib.parse import urlparse
from dateparser import parse
from tqdm.notebook import tqdm
import bs4
import pickle as pkl

## Get URLs

In [3]:
def parse_issue(url):
    if urlparse(url).netloc == 'pubs.acs.org':
        data = requests.get(url).content
        soup = bs4.BeautifulSoup(data)
        
        issue_urls = ['https://pubs.acs.org'+link['href'] for link in soup.findAll(title='Abstract')]
    else:
        raise ValueError('This website cannot be parsed')
        
    return issue_urls  

In [ ]:
links = {}
for volume in range(142,0,-1):
    print(f'{volume}/142')
    for issue in tqdm(range(1, 52)):
        try:
            links[f'{volume}/{issue}'] = parse_issue(f'https://pubs.acs.org/toc/jacsat/{volume}/{issue}')
        except Exception as e:
            print(e)

KeyboardInterrupt: ignored

In [ ]:
with open("links.pkl", 'wb') as f:
    pkl.dump(links, f)

In [ ]:
parse_issue('https://pubs.acs.org/toc/jacsat/142/19')

['https://pubs.acs.org/doi/abs/10.1021/jacs.0c00270',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c03438',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.9b13578',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c00751',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c00755',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c01173',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c01716',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c02007',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c02513',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c02555',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c02892',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c02966',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c03202',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c03537',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.0c04140',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.9b13218',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.9b13309',
 'https://pubs.acs.org/doi/abs/10.1021/jacs.9b13410',
 'https://pubs.acs.org/doi/a

## Parse articles for metadata

In [4]:
def parse_article(url):
    def parse_ints(string):
        if string == '-':
            return 0
        else:
            return int(string)
        
    if urlparse(url).netloc == 'pubs.acs.org':
        article_data = {}

        data = requests.get(url).content
        soup = bs4.BeautifulSoup(data)

        article_data['title'] = soup.find(class_='hlFld-Title').text
        article_data['abstract'] = soup.find(class_='articleBody_abstractText').text
        article_data['doi'] = soup.find(title='DOI URL').text[16:]

        article_data['authors'] = []
        for author in soup.findAll(class_='hlFld-ContribAuthor'):
            author_data = {}

            name = author.find(class_ = 'loa-info-name')
            if name:
                author_data['name'] = name.text

            author_data['affilations'] = [aff.text for aff in author.findAll(class_='loa-info-affiliations-info')]

            orcid = author.find(title='Orcid link')
            if orcid:
                author_data['orcid'] = orcid.text[17:]

            article_data['authors'].append(author_data)

        article_data['date'] = parse(soup.find(class_='pub-date-value').text)

        metric_data = [metric.text for metric in soup.findAll(class_='articleMetrics-val')]
        article_data['views'] = parse_ints(metric_data[0])
        article_data['citations'] = parse_ints(metric_data[2])
    else:
        raise ValueError('This website cannot be parsed')
        
    return article_data

In [7]:
with open('links.pkl', 'rb') as f:
    data = pkl.load(f)

In [17]:
from tqdm.notebook import tqdm
metadatas = []

for k in tqdm(data.keys()):
    for i in tqdm(range(len(data[k]))):
        try:
            metadata = parse_article(data[k][i])
            metadatas.append(metadata)
        except AttributeError:
            print('gotcha attr_error')

  0%|          | 0/820 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

...

  0%|          | 0/67 [00:00<?, ?it/s]

gotcha attr_error


  0%|          | 0/65 [00:00<?, ?it/s]

gotcha attr_error


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [18]:
with open('metadatas.pkl', 'wb') as g:
    pkl.dump(metadatas, g)